In [1]:
import os
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://security.ubuntu.c

In [2]:
from pyspark.sql import SparkSession
!wget https://jdbc.postgresql.org/download/postgresql-42.2.25.jre7.jar
spark = SparkSession.builder.appName('segment1').config('spark.driver.extraClassPath', '/content/postgresql-42.2.25.jre7.jar').config('spark.driver.memory', '100g').getOrCreate()

--2022-05-20 16:16:29--  https://jdbc.postgresql.org/download/postgresql-42.2.25.jre7.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984862 (962K) [application/java-archive]
Saving to: ‘postgresql-42.2.25.jre7.jar’

postgresql-42.2.25. 100%[===================>] 961.78K  --.-KB/s    in 0.1s    

2022-05-20 16:16:29 (6.44 MB/s) - ‘postgresql-42.2.25.jre7.jar’ saved [984862/984862]



In [3]:
from pyspark import SparkFiles
url = 'https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseof_all.csv'
spark.sparkContext.addFile(url)
pseof_df = spark.read.csv(SparkFiles.get('pseof_all.csv'), sep=',', header=True)
url = 'https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseoe_all.csv'
spark.sparkContext.addFile(url)
pseoe_df = spark.read.csv(SparkFiles.get('pseoe_all.csv'), sep=',', header=True)

In [4]:
pseof_df.show(20)
pseoe_df.show(20)

+--------------+----------+-----------+------------+---------+-------+-----------+-----------------+---------+---------+---------+--------+------------+--------------------+------------+--------------------+-------------+---------------------+------------+------------+-------------+-------------------+---------------------------+-------------------+---------------------------+--------------------+----------------------------+-------------------+-------------------+--------------------+
|agg_level_pseo|inst_level|institution|degree_level|cip_level|cipcode|grad_cohort|grad_cohort_years|geo_level|geography|ind_level|industry|y1_grads_emp|y1_grads_emp_instate|y5_grads_emp|y5_grads_emp_instate|y10_grads_emp|y10_grads_emp_instate|y1_grads_nme|y5_grads_nme|y10_grads_nme|status_y1_grads_emp|status_y1_grads_emp_instate|status_y5_grads_emp|status_y5_grads_emp_instate|status_y10_grads_emp|status_y10_grads_emp_instate|status_y1_grads_nme|status_y5_grads_nme|status_y10_grads_nme|
+--------------+--

In [5]:
pseof_df = pseof_df.na.drop(how='any')

In [6]:
pseof_df.show()
pseof_df.select('cip_level').count()

+--------------+----------+-----------+------------+---------+-------+-----------+-----------------+---------+---------+---------+--------+------------+--------------------+------------+--------------------+-------------+---------------------+------------+------------+-------------+-------------------+---------------------------+-------------------+---------------------------+--------------------+----------------------------+-------------------+-------------------+--------------------+
|agg_level_pseo|inst_level|institution|degree_level|cip_level|cipcode|grad_cohort|grad_cohort_years|geo_level|geography|ind_level|industry|y1_grads_emp|y1_grads_emp_instate|y5_grads_emp|y5_grads_emp_instate|y10_grads_emp|y10_grads_emp_instate|y1_grads_nme|y5_grads_nme|y10_grads_nme|status_y1_grads_emp|status_y1_grads_emp_instate|status_y5_grads_emp|status_y5_grads_emp_instate|status_y10_grads_emp|status_y10_grads_emp_instate|status_y1_grads_nme|status_y5_grads_nme|status_y10_grads_nme|
+--------------+--

18843

In [7]:
pseoe_df = pseoe_df.na.drop(how='any')
pseoe_df.show()
pseoe_df.select('cip_level').count()

+--------------+----------+-----------+------------+---------+-------+-----------+-----------------+---------+---------+---------+--------+---------------+---------------+---------------+-------------+---------------+---------------+---------------+-------------+----------------+----------------+----------------+--------------+--------------+--------------+---------------+------------------+--------------------+------------------+--------------------+-------------------+---------------------+---------------------+---------------------+----------------------+
|agg_level_pseo|inst_level|institution|degree_level|cip_level|cipcode|grad_cohort|grad_cohort_years|geo_level|geography|ind_level|industry|y1_p25_earnings|y1_p50_earnings|y1_p75_earnings|y1_grads_earn|y5_p25_earnings|y5_p50_earnings|y5_p75_earnings|y5_grads_earn|y10_p25_earnings|y10_p50_earnings|y10_p75_earnings|y10_grads_earn|y1_ipeds_count|y5_ipeds_count|y10_ipeds_count|status_y1_earnings|status_y1_grads_earn|status_y5_earnings|s

41959

In [8]:
pseof_df.columns

['agg_level_pseo',
 'inst_level',
 'institution',
 'degree_level',
 'cip_level',
 'cipcode',
 'grad_cohort',
 'grad_cohort_years',
 'geo_level',
 'geography',
 'ind_level',
 'industry',
 'y1_grads_emp',
 'y1_grads_emp_instate',
 'y5_grads_emp',
 'y5_grads_emp_instate',
 'y10_grads_emp',
 'y10_grads_emp_instate',
 'y1_grads_nme',
 'y5_grads_nme',
 'y10_grads_nme',
 'status_y1_grads_emp',
 'status_y1_grads_emp_instate',
 'status_y5_grads_emp',
 'status_y5_grads_emp_instate',
 'status_y10_grads_emp',
 'status_y10_grads_emp_instate',
 'status_y1_grads_nme',
 'status_y5_grads_nme',
 'status_y10_grads_nme']

In [9]:
from pyspark.sql.functions import col
cols_to_be_converted_to_int = ['agg_level_pseo',
 'institution',
 'degree_level',
 'cipcode',
 'grad_cohort',
 'grad_cohort_years',
 'geography',
 'industry',
 'y1_grads_emp',
 'y1_grads_emp_instate',
 'y5_grads_emp',
 'y5_grads_emp_instate',
 'y10_grads_emp',
 'y10_grads_emp_instate',
 'y1_grads_nme',
 'y5_grads_nme',
 'y10_grads_nme',
 'status_y1_grads_emp',
 'status_y1_grads_emp_instate',
 'status_y5_grads_emp',
 'status_y5_grads_emp_instate',
 'status_y10_grads_emp',
 'status_y10_grads_emp_instate',
 'status_y1_grads_nme',
 'status_y5_grads_nme',
 'status_y10_grads_nme']
test = pseof_df
for column in cols_to_be_converted_to_int:
  test = test.withColumn(column, col(column).cast('int'))
pseof_df = test

In [10]:
pseof_df.printSchema()

root
 |-- agg_level_pseo: integer (nullable = true)
 |-- inst_level: string (nullable = true)
 |-- institution: integer (nullable = true)
 |-- degree_level: integer (nullable = true)
 |-- cip_level: string (nullable = true)
 |-- cipcode: integer (nullable = true)
 |-- grad_cohort: integer (nullable = true)
 |-- grad_cohort_years: integer (nullable = true)
 |-- geo_level: string (nullable = true)
 |-- geography: integer (nullable = true)
 |-- ind_level: string (nullable = true)
 |-- industry: integer (nullable = true)
 |-- y1_grads_emp: integer (nullable = true)
 |-- y1_grads_emp_instate: integer (nullable = true)
 |-- y5_grads_emp: integer (nullable = true)
 |-- y5_grads_emp_instate: integer (nullable = true)
 |-- y10_grads_emp: integer (nullable = true)
 |-- y10_grads_emp_instate: integer (nullable = true)
 |-- y1_grads_nme: integer (nullable = true)
 |-- y5_grads_nme: integer (nullable = true)
 |-- y10_grads_nme: integer (nullable = true)
 |-- status_y1_grads_emp: integer (nullable =

In [11]:
pseoe_df.columns

['agg_level_pseo',
 'inst_level',
 'institution',
 'degree_level',
 'cip_level',
 'cipcode',
 'grad_cohort',
 'grad_cohort_years',
 'geo_level',
 'geography',
 'ind_level',
 'industry',
 'y1_p25_earnings',
 'y1_p50_earnings',
 'y1_p75_earnings',
 'y1_grads_earn',
 'y5_p25_earnings',
 'y5_p50_earnings',
 'y5_p75_earnings',
 'y5_grads_earn',
 'y10_p25_earnings',
 'y10_p50_earnings',
 'y10_p75_earnings',
 'y10_grads_earn',
 'y1_ipeds_count',
 'y5_ipeds_count',
 'y10_ipeds_count',
 'status_y1_earnings',
 'status_y1_grads_earn',
 'status_y5_earnings',
 'status_y5_grads_earn',
 'status_y10_earnings',
 'status_y10_grads_earn',
 'status_y1_ipeds_count',
 'status_y5_ipeds_count',
 'status_y10_ipeds_count']

In [12]:
cols_to_be_converted_to_int = ['agg_level_pseo',
 'institution',
 'degree_level',
 'cipcode',
 'grad_cohort',
 'grad_cohort_years',
 'geography',
 'industry',
 'y1_p25_earnings',
 'y1_p50_earnings',
 'y1_p75_earnings',
 'y1_grads_earn',
 'y5_p25_earnings',
 'y5_p50_earnings',
 'y5_p75_earnings',
 'y5_grads_earn',
 'y10_p25_earnings',
 'y10_p50_earnings',
 'y10_p75_earnings',
 'y10_grads_earn',
 'y1_ipeds_count',
 'y5_ipeds_count',
 'y10_ipeds_count',
 'status_y1_earnings',
 'status_y1_grads_earn',
 'status_y5_earnings',
 'status_y5_grads_earn',
 'status_y10_earnings',
 'status_y10_grads_earn',
 'status_y1_ipeds_count',
 'status_y5_ipeds_count',
 'status_y10_ipeds_count']
test = pseoe_df
for column in cols_to_be_converted_to_int:
  test = test.withColumn(column, col(column).cast('int'))
pseoe_df = test

In [13]:
pseoe_df.printSchema()

root
 |-- agg_level_pseo: integer (nullable = true)
 |-- inst_level: string (nullable = true)
 |-- institution: integer (nullable = true)
 |-- degree_level: integer (nullable = true)
 |-- cip_level: string (nullable = true)
 |-- cipcode: integer (nullable = true)
 |-- grad_cohort: integer (nullable = true)
 |-- grad_cohort_years: integer (nullable = true)
 |-- geo_level: string (nullable = true)
 |-- geography: integer (nullable = true)
 |-- ind_level: string (nullable = true)
 |-- industry: integer (nullable = true)
 |-- y1_p25_earnings: integer (nullable = true)
 |-- y1_p50_earnings: integer (nullable = true)
 |-- y1_p75_earnings: integer (nullable = true)
 |-- y1_grads_earn: integer (nullable = true)
 |-- y5_p25_earnings: integer (nullable = true)
 |-- y5_p50_earnings: integer (nullable = true)
 |-- y5_p75_earnings: integer (nullable = true)
 |-- y5_grads_earn: integer (nullable = true)
 |-- y10_p25_earnings: integer (nullable = true)
 |-- y10_p50_earnings: integer (nullable = true)

In [14]:
from getpass import getpass
mode = "append"
password = getpass('Enter Password')
jdbc_url="jdbc:postgresql://database.cfqtepfdzy8v.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter Password··········


In [15]:
pseof_df.write.jdbc(url=jdbc_url,  table='pseof', mode=mode, properties=config)

In [16]:
pseoe_df.write.jdbc(url=jdbc_url, table='pseoe', mode=mode, properties=config)

In [17]:
jdbcUrl = 'jdbc:postgresql://database.cfqtepfdzy8v.us-east-2.rds.amazonaws.com:5432/postgres'
connectionProperties = {"user":"postgres", 
          "password": 'postgres', 
          "driver":"org.postgresql.Driver"}

In [18]:
pushdown_query = "(select * from pseoe)AS pseoe_alias"
df = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties)

In [19]:
df.show(20)

+--------------+----------+-----------+------------+---------+-------+-----------+-----------------+---------+---------+---------+--------+---------------+---------------+---------------+-------------+---------------+---------------+---------------+-------------+----------------+----------------+----------------+--------------+--------------+--------------+---------------+------------------+--------------------+------------------+--------------------+-------------------+---------------------+---------------------+---------------------+----------------------+
|agg_level_pseo|inst_level|institution|degree_level|cip_level|cipcode|grad_cohort|grad_cohort_years|geo_level|geography|ind_level|industry|y1_p25_earnings|y1_p50_earnings|y1_p75_earnings|y1_grads_earn|y5_p25_earnings|y5_p50_earnings|y5_p75_earnings|y5_grads_earn|y10_p25_earnings|y10_p50_earnings|y10_p75_earnings|y10_grads_earn|y1_ipeds_count|y5_ipeds_count|y10_ipeds_count|status_y1_earnings|status_y1_grads_earn|status_y5_earnings|s